In [1]:
import pandas as pd
import requests as re
from io import BytesIO
from os import path, makedirs

# Produção de habitação de interesse social

Vários indicadores do GT Urbanismo são relacionados à produção e entrega de unidades de habitação de interesse social. O formulário 15 é relacionado ao orçamento utilizado na produção de unidades de habitação de interesse social. O primeiro indicador físico está no formulário 18 e diz respeito à meta de provimento de moradias do PdM 2021-2024. O formulário 20 é relacionado a uma iniciativa específica do PdM, mas os dados de execução estão disponíveis apenas no relatório da Função Habitação. Por último, os formulários 19 e 21 são relacionado, mas não possuem meta no PdM, apenas nos ODS da Agenda 2030.

Posteriormente, um novo documento apresentou várias informações diferentes relacionadas ao orçamento da função habitação, onde está incluso o Programa 3002. Portanto, vamos manter todo o orçamento da função habitação e apresentar os diferentes recortes no Qlik Sense.

## Orçamento previsto/liquidado no Programa 3002

Vamos começar coletando os dados orçamentários do site de execução orçamentária da Secretaria da Fazenda.

In [ ]:
url_orcamento = 'https://orcamento.sf.prefeitura.sp.gov.br/orcamento/uploads/2024/basedadosexecucao_1224.csv'
df_orcamento = pd.read_csv(url_orcamento,
                           sep=';',
                           decimal=',',
                           encoding='latin1')
df_orcamento

In [ ]:
df_orcamento = df_orcamento.loc[df_orcamento['Cd_Funcao']==16]
df_orcamento

## Orçamento regionalizado no Programa 3002

O orçamento é regionalizado apenas na liquidação, então não é possível obter o orçamento previsto por subprefeitura, mas é possível obter o orçamento liquidado.

In [ ]:
url_orcamento_r = 'https://orcamento.sf.prefeitura.sp.gov.br/orcamento/uploads/2024/basedadosDA_1224.csv'
df_orcamento_r = pd.read_csv(url_orcamento_r,
                           sep=';',
                           decimal=',',
                           encoding='latin1')
df_orcamento_r

In [ ]:
df_orcamento_r = df_orcamento_r.loc[df_orcamento_r['CÓDIGO_FUNÇÃO']==16]
df_orcamento_r = df_orcamento_r.loc[df_orcamento_r['ANO_LIQUIDAÇÃO']==2024]
df_orcamento_r

Vamos conferir qual o percentual do orçamento está presente na tabela de detalhamento e regionalizado a nível de subprefeitura.

In [ ]:
df_orcamento_r['VALOR_DETALHAMENTO_AÇÃO'].sum()/df_orcamento['Vl_Liquidado'].sum()

95,3% de detalhamento é um percentual excelente. Vamos checar a regionalização.

In [ ]:
(
    df_orcamento_r
    .assign(regionalizado=df_orcamento_r['SUBPREFEITURA'].str.contains('Supra')==False)
    .groupby('regionalizado')
    ['VALOR_DETALHAMENTO_AÇÃO'].sum()/df_orcamento['Vl_Liquidado'].sum()
)

O percentual de regionalização é de 64,4%. Menor, mas ainda parece ser satisfatório. Vamos manter as informações de regionalização também.

## Cumprimento da Meta 12: Prover 49.000 moradias de interesse social

Esse indicador está disponível de forma regionalizada na página do PdM da Secretaria de Governo Municipal. 

In [ ]:
url_meta_12 = 'https://prefeitura.sp.gov.br/documents/d/planejamento/relatorio-de-final-do-programa-de-metas-2021-2024_marco2025-xlsx'

resp_meta_12 = re.get(url_meta_12)
resp_meta_12.raise_for_status()
xlsx_meta_12 = BytesIO(resp_meta_12.content)
excel_meta_12 = pd.ExcelFile(xlsx_meta_12)

excel_meta_12.sheet_names

In [ ]:
sheet_name = 'regionalização'
df_meta_12 = pd.read_excel(xlsx_meta_12,
                   sheet_name=sheet_name)
df_meta_12

In [ ]:
df_meta_12 = df_meta_12[df_meta_12['Número da Meta']==12]
df_meta_12

## Produção de habitação de interesse social

O indicador do formulário 19 não está ligado diretamente a nenhuma meta específica do PdM, mas está em consonância com o indicador 11.01.03 dos ODS da Agenda 2030. Os dados desse indicador estão disponíveis no [Observasampa](https://observasampa.prefeitura.sp.gov.br/).

In [ ]:
url_his = 'https://observasampa.prefeitura.sp.gov.br/arquivo.php?cd_indicador=285'
df_his = pd.read_csv(url_his,
                     sep=';',
                     decimal=',',
                     encoding='utf8')
df_his

In [ ]:
df_his = df_his[df_his['nivel_regional']=='Subprefeitura']
df_his

In [ ]:
df_his = df_his.loc[:, [
    'região',
    'indicador',
    '2021',
    '2022',
    '2023'
]]

df_his

In [ ]:
df_his = df_his.melt(
        ['região', 'indicador'],
        ['2021', '2022', '2023'],
        'ano',
        'qtd_unidades'
    )

df_his

## Número de termos de Permissão de Uso (TPU) emitidos em nome da mulher da familia

Esse indicador possui é um exemplo da transversalidade de gênero, que integra a perspectiva de gênero na construção da politica pública voltada à provisão de habitação de interesse social. Os dados desse indicador estão disponíveis, de maneira não regionalizada, no [Observasampa](https://observasampa.prefeitura.sp.gov.br/), no indicador 05.0a.04. Posteriormente, os dados regionalizados estarão presentes no relatório da Função Habitação.

In [ ]:
url_tpu = 'https://observasampa.prefeitura.sp.gov.br/arquivo.php?cd_indicador=662'
df_tpu = pd.read_csv(url_tpu,
                     sep=';',
                     decimal=',',
                     encoding='utf8')
df_tpu

In [ ]:
df_tpu = df_tpu.loc[:, [
    'indicador',
    '2021',
    '2022',
    '2023'
]]

df_tpu

In [ ]:
df_tpu = df_tpu.melt(
        ['indicador'],
        ['2021', '2022', '2023'],
        'ano',
        'qtd_termos'
    )

df_tpu

# Exportando os arquivos

Neste notebook, vamos apenas salvar os arquivos extraídos na pasta de entrada de dados.

In [18]:
output_dir = path.join('dados', 'urbanismo')

if not path.exists(output_dir):
    makedirs(output_dir)

for c, df in [('orcamento_habitacao_original', df_orcamento),
               ('orcamento_regionalizado_habitacao_original', df_orcamento_r),
               ('pdm_meta_12_original', df_meta_12),
               ('his_entregue_original', df_his),
               ('tpu_emitido_original', df_tpu)]:
    filename=path.join(output_dir, c)
    df.to_csv(f'{filename}.csv',
              sep=';',
              decimal=',',
              encoding='utf8',
              index=False
              )